In [3]:
search_for = 126
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0030248165130615234
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 956096141
type: [1, 1, 1, 1, 126] 126
cases of this type: 252047376
100000 36.48182218574045
200000 53.090003267866464
300000 57.97985672636658
400000 52.57823179871972
500000 52.47734905379911
600000 55.5599427913435
700000 57.624906499443206
800000 59.427335168691876
900000 54.96560647168568
1000000 29.922767423915182
1100000 33.53983109342692
1200000 56.04911363677517
1300000 62.874769114496885
1400000 55.52167698956429
1500000 56.79183371375975
1600000 60.92676512102747
1700000 55.4003712308043
1800000 58.01401444613819
1900000 54.42863321043819
2000000 50.42162422656935
2100000 24.05011358444463
2200000 65.050565094112
2300000 55.43815743441706
2400000 48.581041112371686
2500000 63.021239920923485
2600000 64.84011135508038
2700000 57.74272593878828
2800000 43.94122885884719
2900000 58.61054843061133
3000000 44.60102579044

30100000 75.74326887808346
30200000 74.84612864923592
30300000 31.49415738027588
30400000 76.20838562590582
30500000 74.37434436005582
30600000 73.21681356643394
30700000 75.31378035131014
30800000 71.3272658569209
30900000 70.13830326016668
31000000 55.17952121796693
31100000 42.946016965348626
31200000 65.64519791289243
31300000 72.20923028629115
31400000 73.56414472807437
31500000 75.85359549526169
31600000 75.53908283965646
31700000 77.71005664381711
31800000 45.36057578394907
31900000 69.95746277268941
32000000 79.45111917780135
32100000 74.32506734532154
32200000 73.34530978620354
32300000 35.797479905836916
32400000 70.76166383524682
32500000 74.2531936317573
32600000 68.53128355128457
32700000 73.38800128647465
32800000 67.45116089814252
32900000 66.91877401128912
33000000 52.81065945748831
33100000 39.644137596878444
33200000 65.92787780984462
33300000 66.28929667652723
33400000 69.5153664101368
33500000 69.91966873832695
33600000 72.68569944625133
33700000 73.2990564810211
33

60500000 46.594879998693365
60600000 48.98892407778871
60700000 63.1751753727417
60800000 62.59891593982611
60900000 61.95712564355438
61000000 52.59538858457084
61100000 33.304761438576975
61200000 63.586248211643074
61300000 62.929768068119415
61400000 62.63561819440532
61500000 63.11001157957051
61600000 33.96185784563472
61700000 66.37163490343117
61800000 71.91960721352568
61900000 67.90786331998312
62000000 73.26286124598298
62100000 77.74084088415066
62200000 75.5004869787543
62300000 72.73380893067822
62400000 73.1357353124546
62500000 56.140814596530234
62600000 41.438433768761364
62700000 65.5296440396972
62800000 64.5704453909176
62900000 66.46261196791313
63000000 56.33925157258772
63100000 33.12974862712171
63200000 64.93912895037991
63300000 64.08239833530972
63400000 66.25992341150763
63500000 57.724810605759174
63600000 39.100269551837464
63700000 74.2066031893489
63800000 74.79147122816102
63900000 71.58191827413056
64000000 74.79878358243553
64100000 77.04506146455718

90800000 26.02263164240714
90900000 62.63957291565829
91000000 55.56936852856458
91100000 33.13506448695788
91200000 58.46308631336147
91300000 51.60327399200664
91400000 41.394329593851005
91500000 61.51148121448804
91600000 61.676182225950846
91700000 62.805276830721155
91800000 68.6471741845993
91900000 68.26122759562949
92000000 68.97690133997921
92100000 66.85424432149084
92200000 64.4385453981958
92300000 63.21665584450521
92400000 63.60187067142569
92500000 60.30684801585884
92600000 60.92396866178624
92700000 59.21083043288301
92800000 28.17524700253422
92900000 58.98074584936858
93000000 52.85338302842537
93100000 29.196757729420682
93200000 62.53253754991006
93300000 41.62914784318509
93400000 47.33788226213136
93500000 58.62813784704586
93600000 59.28074551665279
93700000 61.89019742555225
93800000 66.1310543436919
93900000 65.10533153817273
94000000 66.83445133000328
94100000 67.8825075745066
94200000 69.33297539175702
94300000 66.0022019730753
94400000 64.55453691198531
94

120300000 72.829803236223
120400000 69.91699733968537
120500000 72.76087457986611
120600000 72.03029289527312
120700000 70.94081042890373
120800000 69.05617174882045
120900000 73.83880859108629
121000000 36.66856997333977
121100000 10.8401595543346
121200000 65.61536826775152
121300000 68.79459443771965
121400000 70.18317642274192
121500000 72.5329018222444
121600000 71.24368523510536
121700000 70.8642911623667
121800000 73.81602983226372
121900000 73.7910821588893
122000000 79.20696946036743
122100000 24.87766147509115
122200000 6.871264225619005
122300000 7.0011081914811815
122400000 6.383096285704606
122500000 6.2174977769182505
122600000 6.193464575250009
122700000 6.378396791659446
122800000 6.790657190578465
122900000 7.005674891959738
123000000 6.854054070533584
123100000 7.51592789709597
123200000 6.9077506391619945
123300000 6.951859024460152
123400000 7.1114322779360775
123500000 6.554317584592951
123600000 6.090608974275716
123700000 6.18358625914694
123800000 6.142562218506

...

884800000 3.4716595362428966
884900000 2.1081905876794576
885000000 3.0829776731001908
885100000 2.459755373336248
885200000 3.267060286622
885300000 3.0204588012886733
885400000 2.3214054163030617
885500000 3.520444005109608
885600000 2.101043030424131
885700000 3.3722081134399278
885800000 1.6816499600756447
885900000 3.349896699465693
886000000 2.146133734480385
886100000 2.9511660651317477
886200000 2.5377771321250506
886300000 2.552481161372315
886400000 4.339223296516621
886500000 2.393504337002423
886600000 1.6811935782720226
886700000 0.5174518658034325
886800000 0.5159707808079476
886900000 0.5233167298279265
887000000 0.5112781820580422
887100000 0.5199229334601687
887200000 0.5152162058891455
887300000 0.5168565618249416
887400000 0.520943464773496
887500000 0.5383317384748234
887600000 0.6616284661559105
887700000 2.9666466971862473
887800000 2.4007936345354564
887900000 4.167141747192281
888000000 2.39487242268105
888100000 3.2634015033376453
888200000 2.2255470252558096
8

best so far: 0
type: [1, 3, 7, 1, 6] 126
cases of this type: 190512
909300000 0.8530014096310946
909400000 0.35605339051023654
best so far: 0
type: [1, 3, 7, 2, 3] 126
cases of this type: 95256
909500000 0.33313983461882474
best so far: 0
type: [1, 3, 7, 3, 2] 126
cases of this type: 63504
909600000 0.3325686872024384
best so far: 0
type: [1, 3, 7, 6, 1] 126
cases of this type: 31752
best so far: 0
type: [1, 3, 14, 1, 3] 126
cases of this type: 47628
909700000 0.32258528715316986
best so far: 0
type: [1, 3, 14, 3, 1] 126
cases of this type: 15876
best so far: 0
type: [1, 3, 21, 1, 2] 126
cases of this type: 21168
best so far: 0
type: [1, 3, 21, 2, 1] 126
cases of this type: 10584
best so far: 0
type: [1, 3, 42, 1, 1] 126
cases of this type: 5292
best so far: 0
type: [1, 6, 1, 1, 21] 126
cases of this type: 1166886
909800000 0.7914402054852089
909900000 1.3768291405215476
910000000 1.3277799371517884
910100000 1.2389339864572584
910200000 1.192165979983878
910300000 0.38502138284307125


921100000 1.1987725053206562
921200000 0.7787727494546771
921300000 1.2486632609701276
921400000 0.9041583089053074
921500000 0.9942782474499008
921600000 1.1955675131303065
921700000 0.8075312549389044
921800000 1.2603452975546876
921900000 1.008789341885851
922000000 1.260696039769962
922100000 0.391829406515193
922200000 0.24719970146162312
922300000 0.25103790455102387
922400000 0.2510914530934076
922500000 0.17987086196988158
922600000 0.21305443621138337
922700000 0.20490262493915162
922800000 0.24840615106935696
922900000 0.20849035411228603
923000000 0.22733643460780978
923100000 0.2488850484868599
923200000 0.2387083175349977
923300000 0.7327080030362962
923400000 0.9460221296889828
923500000 1.3602896131837956
923600000 1.1740155811694999
923700000 0.7240021092280964
923800000 1.0854533928329877
923900000 0.7507265438242615
924000000 0.8476753936509371
924100000 1.199778748472778
924200000 0.7979933276313126
924300000 1.2222292676223603
924400000 0.8007604189243609
924500000 

944800000 0.1711181249482718
944900000 0.10929347749816576
best so far: 0
type: [2, 3, 1, 7, 3] 126
cases of this type: 83349
945000000 0.10769473339095978
best so far: 0
type: [2, 3, 1, 21, 1] 126
cases of this type: 27783
best so far: 0
type: [2, 3, 3, 1, 7] 126
cases of this type: 64827
945100000 0.1049108924724102
best so far: 0
type: [2, 3, 3, 7, 1] 126
cases of this type: 9261
best so far: 0
type: [2, 3, 7, 1, 3] 126
cases of this type: 11907
best so far: 0
type: [2, 3, 7, 3, 1] 126
cases of this type: 3969
best so far: 0
type: [2, 3, 21, 1, 1] 126
cases of this type: 1323
best so far: 0
type: [2, 7, 1, 1, 9] 126
cases of this type: 45927
best so far: 0
type: [2, 7, 1, 3, 3] 126
cases of this type: 15309
945200000 0.09793089588766891
best so far: 0
type: [2, 7, 1, 9, 1] 126
cases of this type: 5103
best so far: 0
type: [2, 7, 3, 1, 3] 126
cases of this type: 5103
best so far: 0
type: [2, 7, 3, 3, 1] 126
cases of this type: 1701
best so far: 0
type: [2, 7, 9, 1, 1] 126
cases of th

955400000 0.008554611345273256
955500000 0.005717493150925636
best so far: 0
type: [6, 1, 1, 3, 7] 126
cases of this type: 64827
best so far: 0
type: [6, 1, 1, 7, 3] 126
cases of this type: 27783
955600000 0.0051694063687509955
best so far: 0
type: [6, 1, 1, 21, 1] 126
cases of this type: 9261
best so far: 0
type: [6, 1, 3, 1, 7] 126
cases of this type: 21609
best so far: 0
type: [6, 1, 3, 7, 1] 126
cases of this type: 3087
best so far: 0
type: [6, 1, 7, 1, 3] 126
cases of this type: 3969
best so far: 0
type: [6, 1, 7, 3, 1] 126
cases of this type: 1323
best so far: 0
type: [6, 1, 21, 1, 1] 126
cases of this type: 441
best so far: 0
type: [6, 3, 1, 1, 7] 126
cases of this type: 7203
best so far: 0
type: [6, 3, 1, 7, 1] 126
cases of this type: 1029
best so far: 0
type: [6, 3, 7, 1, 1] 126
cases of this type: 147
best so far: 0
type: [6, 7, 1, 1, 3] 126
cases of this type: 567
best so far: 0
type: [6, 7, 1, 3, 1] 126
cases of this type: 189
best so far: 0
type: [6, 7, 3, 1, 1] 126
cases 